# Kata for Makemore pt. 3

## Importing packages

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

## Importing data

In [1]:
words = open('../data/names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

## Build vocabulary and map to/from integers

## Build the dataset splits

## Modules

In [5]:
# Linear layer
class Linear:

    def __init__(self, fan_in, fan_out, bias=True):

    def __call__(self, x):

    def parameters(self):
        

# Batch normalisation layer
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):

    def __call__(self, x):

    def parameters(self):

# Tanh
class Tanh:

    def __call__(self, x):
        out = torch.tanh(x)
        return self.out

    def parameters(self):
        return []

IndentationError: expected an indented block after function definition on line 4 (311447316.py, line 6)

## Instantiate the Neural Network

## Training loop

## Diagnostic tools

### Activation distribution

### Gradient distribution

### Weights gradient activation

### Weights update ratio